In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import json
from datetime import datetime
import pysmile
import sys
sys.path.append('/src/python_classes')
import pysmile_license
!jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import *
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
import folium
from colormap import rgb2hex
from folium.plugins import FloatImage
import pickle

from BNModel import BNModel


Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [13]:
a_file = open("Submodels/SM4_water_level/ocean_model.pkl", "rb")
ocean_model_dict = pickle.load(a_file)

a_file = open("Submodels/SM4_water_level/lagoon_model.pkl", "rb")
lagoon_model_dict = pickle.load(a_file)


In [14]:
net.load("")

{'variables': {'wind_u': {'label': 'Wind u vector (m/s)',
   'discretisation': {'n_bins': 5,
    'strategy': 'kmeans',
    'bin_names': ['VeryLow', 'Low', 'Mid', 'High', 'VeryHigh']},
   'child_nodes': ['TWL_less_Tide'],
   'training_data_preprocessed': {0: array([-4.2719574 , -4.07846069, -5.93629456, ..., -8.30755615,
            1.69932556, -0.80697632])},
   'testing_data_preprocessed': {0: array([-4.42945862, -6.86306763, -3.78572083, ...,  7.75259399,
           -2.20835876, -3.63598633])},
   'training_data': {0: array(['Low', 'Low', 'VeryLow', ..., 'VeryLow', 'High', 'High'],
          dtype='<U21')},
   'testing_data': {0: array(['Low', 'VeryLow', 'Low', ..., 'VeryHigh', 'Mid', 'Low'],
          dtype='<U32')},
   'bin_edges': {0: array([-12.8588562 ,  -5.72643208,  -3.45901471,  -0.82166763,
             2.71004547,  12.9216156 ])},
   'id': 0,
   'evidence': [0, 0, 1, 0, 0],
   'resulting_probs': {0: {'VeryLow': 0.0,
     'Low': 0.0,
     'Mid': 1.0,
     'High': 0.0,
     '

In [6]:
# load in the reef and shoreline profile information
df_ocean_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_outer_reef_xyxy_processed.txt')
df_lagoon_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_inner_lagoon_xyxy.txt',delim_whitespace=True,header=None)
df_lagoon_profiles.columns = ['reef_long','reef_lat','shore_long','shore_lat','reef_depth']



In [7]:
def bin_locator(value,bin_edges):
    '''
    function used for determining the index of the appropriate bin for a numerical value.
    '''
    i=0
    for edge_1,edge_2 in zip(bin_edges[:-1],bin_edges[1:]):
        if (value>edge_1)&(value<=edge_2):
            loc_bin = i
        else:
            i+=1
            continue

    if value<=bin_edges[0]:
        loc_bin = 0

    if value>=bin_edges[-1]:
        loc_bin = len(bin_edges)-2

    return(loc_bin)

def model_location(model_dict,location_details,evidence_dict,variable_list):
    
    '''
    function for adding the location information for one side model to the evidence dictionary
    '''
    
    for variable in variable_list:

        bin_edges = model_dict['variables'][variable]['bin_edges'][0]
        value = location_details[variable]

        var_bin = bin_locator(value,bin_edges)
        
        evidence_array = [0]*(len(bin_edges)-1)
        evidence_array[var_bin] = 1
        
        evidence_dict.update({
            variable:evidence_array
        })

    # Add evidence to model dict
    model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)
    
    # Set evidence and get beliefs
    model_location_dict = BNModel().update_evidence(model_location_dict)
    
    return(model_location_dict)

def location_probabilities(evidence_dict,model_dict,variable_list,df_profiles):
    '''
    
    Function for setting evidence and determing probabilties for twl at each point around the island based 
    on the reef characteristics at each location
    
    '''
    
    location_probabilities_dict = {}
#     figure_dict = {}
    
    for index,row in df_profiles.iterrows():

        model_location_dict = model_location(model_dict,row,evidence_dict,variable_list)
        location_probabilities = model_location_dict['variables']['TWL']['resulting_probs'][0]

        df_location_probabilities = pd.DataFrame.from_dict(location_probabilities,orient='index')
        
        largest_cat = df_location_probabilities.idxmax()[0]

        location_probabilities_dict.update({
            (row.reef_long,row.reef_lat):\
                model_dict['variables']['TWL']['discretisation']['bin_names'].index(largest_cat)
        })
        
#         figure_dict.update({
#             (row.reef_long,row.reef_lat):fig#html_graph
#         })
        
    return(location_probabilities_dict)

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"])),
                            properties=dict(name=X["most_likely_twl"])))
    df.apply(insert_features, axis=1)
        
    return(geojson.FeatureCollection(features))

# Load SLR Projections
data_location = "/src/Dataset/D7_MSL_projections/"
file_name = "distributions_dict"
with open("{}{}.json".format(data_location,file_name), 'r') as fp:
    SL_proj_dict = json.load(fp)
    
def SLR_proj_extractor(SL_proj_dict,AIS_config,rcp,year):
    '''
    Function for getting SLR projections for a given Antarctic icesheet, rcp and year
    Years start as 2020 and go up in lots of 10 until 2150 (2100 for dp16)
    '''
    SLR_prob_dict = SL_proj_dict["('{}', '{}', {})".format(AIS_config,rcp,year)]
    SLR_median_prob = np.max([float(x) for x in list(SLR_prob_dict.keys())])
    SLR_median_MSL = float(SLR_prob_dict[str(SLR_median_prob)])/1000 #units is m

    return(SLR_median_MSL)

In [8]:
def figure(
    view,tide_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin,proj_time
    ):
    
    ########### get the MSL bin based on slider value
    msl_proj = SLR_proj_extractor(SL_proj_dict,'k14','26','{}'.format(proj_time))
    ocean_model_dict['variables']['MSL']['bin_edges'][0]
    bin_count = bin_locator(msl_proj,ocean_model_dict['variables']['MSL']['bin_edges'][0])
    msl_bin = ocean_model_dict['variables']['MSL']['discretisation']['bin_names'][bin_count]
    
    if view == 'Map':
        map_osm = folium.Map(location=[1.448888, 172.991794],zoom_start=11)
    elif view == 'Satellite':
        token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
        tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

        map_osm = folium.Map(location=[1.448888, 172.991794], zoom_start=11, tiles=tileurl, attr='Mapbox')
        
    twl_bin_edges = [round(x,2) for x in ocean_model_dict['variables']['TWL']['bin_edges'][0]]
    twl_bins = ocean_model_dict['variables']['TWL']['discretisation']['bin_names']

    colours_rgb = matplotlib.pyplot.get_cmap('seismic')(np.arange(0,1+1/len(twl_bins),1/(len(twl_bins)-1)))
    colour_hex_dict = {i:rgb2hex(int(255*colours_rgb[i][0]),int(255*colours_rgb[i][1]),int(255*(colours_rgb[i][2]))) for i in np.arange(0,len(twl_bins),1)}
    
    ################################################
    
    # Create an empty dictionary for the evidence and populate as you go
    ocean_evidence_dict = {}
    
    for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin],
                                ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore']):

        ## Set in the evidence dict to be as indicated in the dropdown
        bin_index = ocean_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
        # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
        evidence = [0 for x in ocean_model_dict['variables'][var_name]['discretisation']['bin_names']]
        evidence[bin_index] = 1
        ocean_evidence_dict.update({
            var_name:evidence
        })
    
    # Create a list of variables that are location specific to set as evidence in the network
    variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']
    
    # get the probability dictionary
    location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)
    
    # Create dataframe to plot
    df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
    df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
    df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
    df_twl_locations.reset_index(drop=True,inplace=True)
    
    data_ocean = data2geojson(df_twl_locations)
    
    colors_hex_points_ocean = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
    #####################################################################
    
    # Create an empty dictionary for the evidence and populate as you go
    lagoon_evidence_dict = {}
    
    for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin],
                                ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore','wind_u','wind_v']):

        ## Set in the evidence dict to be as indicated in the dropdown
        bin_index = lagoon_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
        # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
        evidence = [0 for x in lagoon_model_dict['variables'][var_name]['discretisation']['bin_names']]
        evidence[bin_index] = 1
        lagoon_evidence_dict.update({
            var_name:evidence
        })
    
    # Create a list of variables that are location specific to set as evidence in the network
    variable_list = []
    
    # get the probability dictionary
    location_probabilities_dict = location_probabilities(lagoon_evidence_dict,lagoon_model_dict,variable_list,df_lagoon_profiles)
    
    # Create dataframe to plot
    df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
    df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
    df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
    df_twl_locations.reset_index(drop=True,inplace=True)
    
    data_lagoon = data2geojson(df_twl_locations)
    
    colors_hex_points_lagoon = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
    #####################################################################
    
    features_list = data_ocean['features']+data_lagoon['features']
    
    data = data_ocean
    data.update({
        'features':features_list
    })
    
    colors_hex_points = colors_hex_points_ocean+colors_hex_points_lagoon
    
    #####################################################################

    for feature,color in zip(features_list,colors_hex_points):
        feature['properties'] = {'color':color,'weight':1,'markerColor':color,'fillOpacity':1,'fillColor':color}
        long,lat = feature['geometry']['coordinates']
        
        marker = folium.CircleMarker([lat,long],color=color,
                                    # popup='<img src={}_{}.png>'.format(int(long*1000),int(lat*1000)),
                                   fill_color=color,fill=True,fill_opacity='1',radius=5)
        marker.add_to(map_osm)
        
    twl_bin_edge_labels = ['{} to {} m'.format(
        x,y) for x,y in zip(twl_bin_edges[:-1],twl_bin_edges[1:])]
        
    output_list = []
    for rgb_color in colours_rgb:
        output = plt.scatter([],[],color=rgb_color)
        output_list.append(output)
        
    legend = plt.legend(output_list,twl_bin_edge_labels,title='Total water level anomaly',fontsize=10)
    plt.setp(legend.get_title(),fontsize=12)
    
    plt.axis('off')
    plt.savefig('legend.png')
    
    plt.close()
    
    url = (
        "legend.png"
    )    
    
    FloatImage(url, bottom=55, left=55).add_to(map_osm)
    
    map_osm.save('test.html')
        
    return(map_osm)
    

# compile the figure
# lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
tide_bins = ocean_model_dict['variables']['Tide']['discretisation']['bin_names']
wave_height_bins = ocean_model_dict['variables']['Hs_offshore']['discretisation']['bin_names']
wave_period_bin = ocean_model_dict['variables']['Tm_offshore']['discretisation']['bin_names']
wave_direction_bin = ocean_model_dict['variables']['Dir_offshore']['discretisation']['bin_names']
wind_u_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
wind_v_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# time = list(model_dicts_through_time_dict.keys())  


# Create the plot with the widget
map_osm = interact(figure,
                view = widgets.Dropdown(options=['Map','Satellite'],value='Map',description='View type',disabled=False),
                tide_bin = widgets.Dropdown(options=tide_bins,value='Mid',description='Tide',disabled=False),
                wave_height_bin = widgets.Dropdown(options=wave_height_bins,value='Mid',description='Wave height',disabled=False),
                wave_period_bin = widgets.Dropdown(options=wave_period_bin,value='Mid',description='Wave period',disabled=False),
                wave_direction_bin = widgets.Dropdown(options=wave_direction_bin,value='NNE',description='Wave direction',disabled=False),
                wind_u_bin = widgets.Dropdown(options=wind_u_bin,value='Mid',description='Wind u',disabled=False),
                wind_v_bin = widgets.Dropdown(options=wind_v_bin,value='Mid',description='Wind v',disabled=False),
                proj_time = widgets.IntSlider(min=2020,max=2150,step=10,value=2020,description='SLR prediction')
               )


map_osm

NameError: name 'ocean_model_dict' is not defined